# Deploy the model

In the previous part of the lab, you fined tuned hyper-parameters of your model and registered the best performing version in your workspace.

Now, you're ready to deploy the model.

You can deploy the model to a variety of target runtimes. In this part of the lab you wiil deploy the model as a web service in Azure Container Instance.

![AML Arch](images/amlarch.png)

## Retrieve the model
You registered a model in your workspace in the previous tutorial. Now, load this workspace and download the model to your local directory.


In [18]:
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model=Model(ws, 'aerial_classifier')
model.download(target_dir = '.')
import os 
# verify the downloaded model file
os.stat('./model.hd5')

Found the config file in: /data/home/demouser/notebooks/AIDays/DataScientistTrack/02-AML-EndToEndWalkthrough/aml_config/config.json


os.stat_result(st_mode=33204, st_ino=5912883, st_dev=2081, st_nlink=1, st_uid=1003, st_gid=1003, st_size=12648160, st_atime=1539149070, st_mtime=1539149071, st_ctime=1539149071)

## Deploy as web service

To build the correct environment for ACI, provide the following:
* A scoring script that invokes the model
* An environment file to show what packages need to be installed
* A configuration file to build the ACI
* The model you trained before

### Create scoring script

Create the scoring script, called score.py, used by the web service call to invoke the model.

You must include two required functions in the scoring script:
* The `init()` function, which loads the model into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats can be used.


In [19]:
%%writefile score.py
import json
import os
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
import numpy as np
import random
from azureml.core.model import Model

def init():
    # Instantiate Resnet50 featurizer
    global featurizer
    featurizer = resnet50.ResNet50(
            weights = 'imagenet', 
            input_shape=(224,224,3), 
            include_top = False,
            pooling = 'avg')

    # Load the model
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('aerial_classifier')
    model = tf.keras.models.load_model(model_path)

def run(raw_data):
    # convert json to numpy array
    img = np.array(json.loads(raw_data)['data'])
    # img = np.expand_dims(img, axis=0)
    # normalize as required by ResNet50
    # img = resnet50.preprocess_input(img)
    # extract bottleneck features
    # features = featurizer.predict(img)
    # make prediction
    # predictions = model.predict(features)
    predictions = img.shape
    return json.dumps(predictions.tolist())

Overwriting score.py


### Create environment file
Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `tensorflow`, `h5py` and `azureml-sdk`.

In [20]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("tensorflow")
myenv.add_conda_package("h5py")
myenv.add_pip_package("pynacl==1.2.1")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of `myenv.yml` file

In [21]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
  - pynacl==1.2.1
- tensorflow
- h5py



### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. The default is 1 core and 1 gigabyte of RAM. Since we are using ResNet50 featurizer we are CPU bound.  In this lab we will the defaults but you experiment with the right configuratin.

In [22]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "aerial",  "method" : "classifier"}, 
                                               description='Predict aerial images')

### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment file (`myenv.yml`)
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

In [23]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='aerial-classifier-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image aerial-classifier-svc:3, operation "Succeeded"
Creating service
Running.......................................................
FailedACI service creation operation finished, operation "Failed"
Service creation failed, unexpected error response:
{'code': 'AciDeploymentFailed', 'message': 'Aci Deployment failed', 'details': [{'code': 'CrashLoopBackOff', 'message': "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance aerial-classifier-svc.\nYou can also try to run image jkamlacrpqevakrf.azurecr.io/aerial-classifier-svc:3 locally. More information at http://aka.ms/debugimage"}]}
CPU times: user 2.2 s, sys: 91 ms, total: 2.29 s
Wall time: 7min 45s


Get the scoring web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [24]:
print(service.scoring_uri)

None


## Test deployed service

You will now test the deployed model with a random sample of 30 images from the test data.  

The following code goes through these steps:
1. Send the data as a JSON array to the web service hosted in ACI. 

2. Use the SDK's `run` API to invoke the service. You can also make raw calls using any HTTP tool such as curl.

3. Print the returned predictions 

 

In [ ]:
import json

# find 30 random samples from test set
n = 30
sample_indices = np.random.permutation(X_test.shape[0])[0:n]

test_samples = json.dumps({"data": X_test[sample_indices].tolist()})
test_samples = bytes(test_samples, encoding = 'utf8')

# predict using the deployed model
result = json.loads(service.run(input_data=test_samples))


## Clean up resources

To keep the resource group and workspace for other tutorials and exploration, you can delete only the ACI deployment using this API call:

In [15]:
service.delete()